In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def get_consent_task():
    return {
        "name": "Consent",
        "type": "InstructionsTask",
        "prerequisite": True,
        "content": {
            "type": "link",
            "url": "$$www$$/consent.md"
        },
        "form": {
            "fields": [
                {
                    "name": "consent1",
                    "label": "I have read and understood the study information dated [DD/MM/YYYY], or it has been read to me. I have been able to ask questions about the study and my questions have been answered to my satisfaction.",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                },
                {
                    "name": "consent2",
                    "label": "I consent voluntarily to be a participant in this study and understand that I can refuse to answer questions and I can withdraw from the study at any time, without having to give a reason. ",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                },
                {
                    "name": "consent3",
                    "label": "I understand that taking part in the study involves providing data ratings or annotations which will be recorded but not linked to my identity, along with data about my interaction with the web interface.",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                },
                {
                    "name": "consent4",
                    "label": "I understand that information I provide will be used for scientific publications.",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                },
                {
                    "name": "consent5",
                    "label": "I understand that personal information collected about me that can identify me, such as [e.g. my name or where I live], will not be asked nor stored.",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                },
                {
                    "name": "consent6",
                    "label": "I give permission for the annotations that I provide to be archived in networked storage at Delft University of Technology in anonymized form so it can be used for future research. I understand that data will not be made public but may be shared with researchers under an End User License Agreement (EULA).",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                }
            ]
        }
}

In [3]:
def get_instructions_task(file, type='video'):
    titles = {
        'video': 'Video-only instructions',
        'audio': 'Audio-only instructions',
        'av': 'Audiovisual instructions'
    }
    return {
        "name": titles[type],
        "type": "InstructionsTask",
        "prerequisite": False,
        "content": {
            "type": "link",
            "url": f"$$www$$/{file}"
        }
    }

In [122]:
def get_rating_tasks(ex, idx, fpath='laughter_examples/av', calibration=False):
    return [
      {
            "type": "Continuous1DTask",
            "name": 'Calibration' if calibration else f"#{idx}: Recognition",
            "media": {
                "type": "video",
                "url": f"$$www$$/{fpath}/{ex.get_file_id()}.mp4",
                "fps": 25
            },
            "intensityInput": {
                "mode": "binary"
            },
            "instructions": "Find the person indicated with a red rectangle. Play the video and press indicate the ocurrence of laughter by pressing \"q\"."
      },
      {
            "type": "QuestionnaireTask",
            "name": 'Intensity' if calibration else f"#{idx}: Rating",
            "form": {
            "fields": [
                {
                  "name": "intensity",
                  "label": "Laughter intensity: ",
                  "required": True,
                  "input": {
                    "inputType": "Slider",
                    "defaultValue": 50
                  }
                },
                {
                  "name": "confidence",
                  "label": "Annotation confidence: ",
                  "required": True,
                  "input": {
                    "inputType": "Slider",
                    "defaultValue": 50
                  }
                }
            ]
            },
            "media": {
                "type": "video",
                "url": f"$$www$$/{fpath}/{ex.get_file_id()}.mp4",
            },
            "instructions": "Find the person indicated with a red rectangle. Play the video and pay attention to this person. Give the required ratings on the right."
      }]

In [7]:
import json
import os
import random
import numpy as np
import pandas as pd
sys.path.append('../lared')
from dataset.example import VideoExample, AudioExample, FullExample
random.seed(22)
np.random.seed(22)

In [74]:
pilot_path = '/home/jose/drive/data/lared_laughter/pilot2'
laughter_examples_df = pd.read_csv(os.path.join(pilot_path, 'laughter_examples', 'examples.csv'), index_col=0)
laughter_examples_df = laughter_examples_df[laughter_examples_df['rect'].notnull()]
speech_examples_df = pd.read_csv(os.path.join(pilot_path, 'speech_examples', 'examples.csv'), index_col=0)
speech_examples_df = speech_examples_df[speech_examples_df['rect'].notnull()]

In [101]:
laughter_examples = [('laughter', FullExample(**ex[1].to_dict())) for ex in laughter_examples_df.iterrows()]
speech_examples = [('speech', FullExample(**ex[1].to_dict())) for ex in speech_examples_df.iterrows()]

In [102]:
obvious_examples_ids = [
    '64a92aea9395ace7ac9d60eab34911e419fc66610cd76f1e29df4b4fd16f230f',
    '0feb2d80d7e192750c723d7f70dd82b61f3e98de2a437542ae48f5408b073973',
    '85aac70ec91eb3be1b313b33e0b7828394bbe4e4edc6a956d1e7061dfc8b250e',
    '68d229cf19eec82f37580265ea93892117dd5b559b04d22489da3593315f18e7',
    'ced6e78fe7940c10fbc9d7c385273e68459ca399ccb668c8123cf5a66fa99819',
    'bb6337eea970487ce9cd4ff26ea78c7acc6d5d1a355b7aa50029a3229f115b21'
]
obvious_examples = [ex[1] for i, ex in enumerate(laughter_examples) if ex[1].get_hash() in obvious_examples_ids]
laughter_examples = [ex for i, ex in enumerate(laughter_examples) if ex[1].get_hash() not in obvious_examples_ids]
len(obvious_examples), len(laughter_examples)

(6, 83)

In [103]:
speech_examples = random.sample(speech_examples, 37)

In [104]:
examples = [*laughter_examples, *speech_examples]
random.shuffle(examples)
len(examples)

120

In [123]:
def get_hit(name, s1, s2, s3, calibration_set):

    s1_tasks = [t for i, ex in enumerate(s1) for t in get_rating_tasks(ex[1], idx=i+1, fpath=f'{ex[0]}_examples/video')]
    s1_tasks = [
        *get_rating_tasks(calibration_set[0], idx=0, fpath='laughter_examples/video', calibration=True),
        *s1_tasks[:40], 
        *get_rating_tasks(calibration_set[1], idx=0, fpath='laughter_examples/video', calibration=True), 
        *s1_tasks[40:]
    ]

    s2_tasks = [t for i, ex in enumerate(s2) for t in get_rating_tasks(ex[1], idx=i+1, fpath=f'{ex[0]}_examples/aiv')]
    s2_tasks = [
        *get_rating_tasks(calibration_set[2], idx=0, fpath='laughter_examples/aiv', calibration=True),
        *s2_tasks[:40], 
        *get_rating_tasks(calibration_set[3], idx=0, fpath='laughter_examples/aiv', calibration=True), 
        *s2_tasks[40:]
    ]

    s3_tasks = [t for i, ex in enumerate(s3) for t in get_rating_tasks(ex[1], idx=i+1, fpath=f'{ex[0]}_examples/av')]
    s3_tasks = [
        *get_rating_tasks(calibration_set[4], idx=0, fpath='laughter_examples/av', calibration=True),
        *s3_tasks[:40], 
        *get_rating_tasks(calibration_set[5], idx=0, fpath='laughter_examples/av', calibration=True), 
        *s3_tasks[40:]
    ]

    return {
        "id": name,
        "name": name,
        "interface": {
            "type": "timeline",
            "showProgress": True
        },
        "tasks": [
            get_consent_task(),

            get_instructions_task('video_instructions.md', 'video'),
            *s1_tasks,

            get_instructions_task('audio_instructions.md', 'audio'),
            *s2_tasks,

            get_instructions_task('av_instructions.md', 'av'),
            *s3_tasks,
        ]
    }

In [124]:
set1 = examples[0:40]
set2 = examples[40:80]
set3 = examples[80:]
project = {
    "name": "Pilot",
    "id": "pilot",
    "email": "example@example.com",
    "hits": [
        get_hit('HIT1', set1, set2, set3, obvious_examples),
        get_hit('HIT2', set2, set3, set1, obvious_examples),
        get_hit('HIT3', set3, set1, set2, obvious_examples)
    ]
}

In [125]:
json.dump(project, open('pilot.covfee.json', 'w'), indent=2)